In [ ]:
#Section 1: load and preprocess raster (remove 4th band if present)

In [ ]:
import rasterio
import numpy as np

# Define the correct raster path
raster_path = [insert source file path]
fixed_raster_path = [insert source file path]
# Open the raster and check the number of bands
with rasterio.open(raster_path) as src:
    print(f" Original raster has {src.count} bands. Expected: 3")

    if src.count > 3:
        print(f" Removing extra bands...")

        # Read only the first 3 bands (Red, Green, Blue)
        data = src.read([1, 2, 3])  # Shape: (3, Height, Width)

        # Update profile for 3-band output
        profile = src.profile
        profile.update(count=3, dtype=data.dtype)

        # Save the new raster
        with rasterio.open(fixed_raster_path, "w", **profile) as dst:
            dst.write(data)

        print(f" Fixed raster saved at: {fixed_raster_path}")
        print(f" Image Shape: {data.shape}")  # Should be (3, Height, Width)
    else:
        print("Raster already has 3 bands. No changes needed.")


In [ ]:
#Downsample orthmosaic to 10cm/pixel for Deepforest predicitons

In [ ]:
import os
import rasterio
from rasterio.enums import Resampling

with rasterio.open(fixed_raster_path) as src:
    scale_factor = 10 / [insert calculated GSD from orthomosaic**]
    new_width = int(src.width / scale_factor)
    new_height = int(src.height / scale_factor)

    data = src.read(
        out_shape=(src.count, new_height, new_width),
        resampling=Resampling.bilinear
    )

    new_transform = src.transform * src.transform.scale(
        src.width / new_width,
        src.height / new_height
    )

    profile = src.profile
    profile.update({
        'height': new_height,
        'width': new_width,
        'transform': new_transform
    })
    
    with rasterio.open("downsampled.tif", "w", **profile) as dst:
        dst.write(data)

print("Downsampled raster saved to: downsampled.tif")
downsampled_path = "downsampled.tif"



In [ ]:
#Section 2: predict tree location using predict_tile().

In [ ]:
from deepforest import main

# Initialize DeepForest
model = main.deepforest()
model.load_model(model_name="weecology/deepforest-tree", revision="main")

# Predict on the downsampled image
predicted_results = model.predict_tile(
    downsampled_path, 
    patch_size=400, 
    patch_overlap= 0.35
)

# Print results
print("\n Predictions Before Conversion:")
print(predicted_results.head())


In [ ]:
# Save to working directory
predicted_results.to_csv("Output.csv", index=False)
print("\n Predictions saved to: Output.csv")


In [ ]:
#Convert x & y min and max coordinates to geographical coordinatees

In [ ]:
from deepforest import utilities
import os

# Define local predictions CSV and raster file in current directory
predictions_path = os.path.abspath("Output.csv")             # Assumes Output.csv is in working directory
raster_path = os.path.abspath("downsampled.tif")             # Local downsampled raster

# Read Predictions
print("\n Reading Predictions CSV...")
df = utilities.read_file(predictions_path)

# Convert to Geographic Coordinates
print("\n Converting to Geographic Coordinates...")
geo_df = utilities.image_to_geo_coordinates(df, root_dir=os.getcwd())

# Print sample to verify conversion
print("\n Sample Converted Predictions:")
print(geo_df.head())



In [ ]:
#Convert predictions to shapefile

In [ ]:
import geopandas as gpd

# Define output path
shapefile_output_path = [insert destination file path]

# Save the converted geographic predictions
geo_df.to_file(shapefile_output_path)

print(f"\n Predictions saved as a shapefile: {shapefile_output_path}")